# Introdução (não-extensiva) a Online Learning


**Saulo Martiello Mastelini** (mastelini@usp.br)

Outras redes:

- [Github](https://github.com/smastelini)
- [Linkedin](https://www.linkedin.com/in/smastelini/) (eu deveria, mas não atualizo frequentemente -- para ser sincero, está muito desatualizado)
- [ResearchGate](https://www.researchgate.net/profile/Saulo-Mastelini)

MBA em Ciência de Dados<br>
Universidade de São Paulo, São Carlos, Brasil<br>
Copyright (c) 2021

---

**Disclaimer**

Como o título já diz, essa não é, de forma alguma, uma introdução extensiva ao tema. É apenas a minha humilde tentativa de dar um panorama geral de décadas de pesquisa em uma área que está em constante expansão e inovação.

---


## Sumário
1. Online learning? Why?
2. Batch vs. Online
3. Estruturas necessárias: exemplo
    - Mean
    - Var
    - Quantile
4. Por que usar dicionários?
    - Exemplos com dados faltantes
5. Como avaliar um modelo?
    - `progressive_val_score`
    - label delay
6. Exemplos de algoritmos
    1. Classificação
        1. Logistic regression
        2. Hoeffding Tree
        3. Naive Bayes
        4. Adaptive Random Forest
        5. Streaming Random Patches
    2. Regressão
        1. Linear Regression
        2. Hoeffding Tree
        3. Stochastic Gradient Trees
        4. AMRules
        5. Adaptive Random Forest
        6. Streaming Random Forest
    3. Clustering
        1. k-Means
        2. CluStream
        3. DenStream
        4. DBSTREAM
    4. Anomaly detection
        1. Half-space Trees
7. Expert module
8. Pipelines e pré-processamento
    - Encoding
    - Scaling
    - Filtragem e Seleção
    - Aritmética com Pipelines
    - Visualizando as coisas
9. Exemplo completo
    - processamento em tempo real
    - inspeção de modelos

# 1. Online Learning? Why?

Por que alguém deveria se preocupar em atualizar modelos a todo tempo? Não é só treinar e sair usando?

R: sim! Em grande parte dos casos isso é verdade.

Mas imagine que:

- A quantidade de dados produzidas é enorme
- Não é possível armazenar tudo
- Existe limitação no poder computacional para treinamento dos modelos
    - processador
    - memória
    - bateria
- Os dados são não-estacionários e/ou evoluem com o tempo

Nesses casos, posso usar aprendizado de máquina tradicional? R: sim!!

É possível usar aprendizado de máquina tradicional se:
- Os dados são estacionários (uma amostra representativa dos dados é suficiente)

ou

- A velocidade de produção dos dados não é tão alta (batch-incremental e os recursos computacionais são suficientes

## 1.1 Batch-incremental


Um modelo tradicional de aprendizado de máquina é re-treinado de tempos em tempos. Para tal, uma janela para treinamento precisa ser definida.

Tipos de janelamento dos dados:


<img src="time_windows.png">

**Fonte:** Adaptado de:

> Carnein, M. and Trautmann, H., 2019. Optimizing data stream representation: An extensive survey on stream clustering algorithms. Business & Information Systems Engineering, 61(3), pp.277-297.

- *Landmarks* são a escolha mais comum em estratégias batch-incremental. É preciso definir o tamanho da janela de forma adequada.
    - O modelo pode ficar defasado se a janela for muito grande 
    - Ou o modelo pode não capturar os padrões se a janela for muito pequena
    - Concept-drift é um problema e tanto
    
**Atenção**: batch-incremental != mini-batch.
Redes neurais podem ser treinadas de forma incremental ou progressiva. No entanto, questões como "esquecimento catastrófico" são problemáticas. Esse e outros tipos de desafios são tratados na área de pesquisa chamada **continual learning**.

## 1.2 Importante lembrar

Data streams não são, necessariamente, séries temporais! 🤯

Mas qual é a diferença, então?

Basicamente, em streams não necessariamente temos dependências temporais explícitas como em séries temporais. Por exemplo: rede de sensores.

Os dados chegam temporalmente organizados, mas cada sensor tem uma taxa de transmissão específica, ou um delay específico. Alguns sensores pode falhar... outros serem adicionados. E assim por diante.

A ordem de chegada não importa... tanto, mas importa.

# 2. Batch vs. Online

Uma boa introdução acerca da "migração" de batch para online está disponível no [site](https://riverml.xyz/latest/examples/batch-to-online/) do River. Aqui eu só quero dar uma visão bem geral das diferenças. Entraremos em mais detalhes posteriormente nas minúcias de avaliação de modelos em Online Learning.


Uma possível validação de um modelo tradicional de aprendizado de máquina poderia ter essa "cara":

In [2]:
from sklearn.datasets import load_wine
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier

data = load_wine()

X, y = data.data, data.target
kf = KFold(shuffle=True, random_state=8, n_splits=10)

accs = []

for train, test in kf.split(X):
    X_tr, X_ts = X[train], X[test]
    y_tr, y_ts = y[train], y[test]
    
    dt  = DecisionTreeClassifier(max_depth=5, random_state=93)
    dt.fit(X_tr, y_tr)
    
    accs.append(accuracy_score(y_ts, dt.predict(X_ts)))

print(f"Acurácia média: {sum(accs) / len(accs)}")

Acurácia média: 0.9045751633986928


Reparem que todo o conjunto de dados está disponível e carregado em memória. O algoritmo de árvore de decisão pode percorrer os dados de treino quantas vezes forem necessárias. Os dados de teste (validação, no nosso caso) nunca são utilizados para treino.

No fim das contas, usaríamos o conjunto completo para treinar um modelo final (supondo que já encontramos um conjunto adequado de hiper-parâmetros). Uma vez treinado, esse modelo seria utilizado para fazer predições para novas amostras (de vinhos, nesse caso).

In [16]:
from river import metrics
from river import stream
from river import tree


acc = metrics.Accuracy()
ht = tree.HoeffdingTreeClassifier(max_depth=5, grace_period=20)

for x, y in stream.iter_sklearn_dataset(load_wine()):
    # Métrica atualizada antes do treinamento
    acc.update(y, ht.predict_one(x))
    # Modelo treinado amostra-a-amostra
    ht.learn_one(x, y)

print(f"Acurácia: {acc.get()}")

Acurácia: 0.9269662921348315


Aqui, estamos percorrendo cada exemplo do conjunto de dados de forma sequencial. Os exemplos poderiam ser carregados diretamente do disco, de algum webservice, ou de onde sua imaginação te levar, sem a necessidade de salvar em memória nada.

Cada amostra é primeiramente utilizada para teste e depois passada para o modelo. Tudo é atualizado uma amostra por vez. Notem que não podemos comparar diretamente os desempenhos obtidos porque a estratégia de avaliação foi diferente nos dois casos.

Poderíamos ainda (pseudo) embaralhar os dados antes de passar para o modelo, se temos plena confiança que os dados são estacionários.

# 3. Estruturas necessárias: exemplo

In [1]:
import numpy as np
from river import stats

Dado esse choque inicial, vamos prosseguir com calma e primeiramente pensar em alguns aspectos importantes antes de vermos os algoritmos de aprendizado de máquina online.

## 3.1. Um exemplo, para aquecimento cerebral

Vamos supor que queremos calcular estatísticas para dados que chegam a todo momento:

- Média
- Variância
- ...

Hora de simular:

In [2]:
import random

rng = random.Random(42)

In [3]:
%%time

valores = []
stds_batch = []

for _ in range(50000):
    v = rng.gauss(5, 3)
    valores.append(v)
    
    stds_batch.append(np.std(valores))

CPU times: user 1min, sys: 27.5 ms, total: 1min
Wall time: 1min


Por padrão o numpy calcula o desvio padrão populacional! Para usar a versão amostral precisamos passar `ddof=1` como parâmetro.

In [4]:
rng = random.Random(42)

In [5]:
%%time

stds_incr = []
var = stats.Var(ddof=0)

for _ in range(50000):
    v = rng.gauss(5, 3)
    var.update(v)
    stds_incr.append(var.get() ** 0.5)

CPU times: user 107 ms, sys: 0 ns, total: 107 ms
Wall time: 106 ms


Será que dá alguma diferença? E será que funciona?

In [6]:
s_erros = 0

for batch, incr in zip(stds_batch, stds_incr):
    s_erros += (batch - incr)

s_erros, s_erros / len(stds_batch)

(4.301100448023121e-10, 8.602200896046241e-15)

Ok... parece convincente. Mas e se o cenário fosse diferente? Se ao invés de calcularmos o desvio padrão e irmos aumentando a quantidade de dados, quisessemos descobrir um percentil das últimas `N` amostras?

In [7]:
rng = random.Random(42)

tam = 10000

In [8]:
%%time


buffer = []
percs75_batch = []

for _ in range(100000):
    v = rng.uniform(-100, 100)
    buffer.append(v)
    if len(buffer) <= tam:
        continue
        
    percs75_batch.append(np.quantile(buffer, q=0.75))
    buffer.pop(0)

CPU times: user 54.1 s, sys: 27.4 ms, total: 54.1 s
Wall time: 54.1 s


Podemos fazer um pouco melhor que isso.

In [11]:
rng = random.Random(42)

In [12]:
%%time

qtl = stats.RollingQuantile(window_size=tam, q=0.75)
percs75_incr = []

for _ in range(100000):
    v = rng.uniform(-100, 100)
    qtl.update(v)
    if len(qtl) <= tam:
        continue
        
    percs75_incr.append(qtl.get())

CPU times: user 5.55 s, sys: 0 ns, total: 5.55 s
Wall time: 5.56 s


In [13]:
s_erros = 0

for batch, incr in zip(percs75_batch, percs75_incr):
    s_erros += (batch - incr)

s_erros

0

Espero tê-los convencido! O [módulo](https://riverml.xyz/dev/api/overview/#stats) `stats` do River tem várias funções úteis para sumarização de dados de forma incremental. Vale a pena checar! 🧐

A razão pela qual eu quis mostrar tudo isso é porque vários desses algoritmos são os "tijolos" que compõem os algoritmos de aprendizado de máquina. Uma receita breve para online learning:

- A junção de métricas estatísticas incrementais
- Algumas grandezas probabilísticas
- Algumas teorias de aprendizado de máquina que são genéricas e não limitadas ao cenário in-batch
- Gradiente descendente
- *otras cositas*

Com isso tudo é possível formar boa parte dos algoritmos de aprendizado incremental. Mas isso é extremamente genérico de se dizer, não é? Bom, no fim das contas, o aprendizado de máquina tradicional também "se resume" a algumas coisas pontuais, mas que são extremamente abrangentes por si só.

Que tal entendermos um pouco de como os exemplos anteriores funcionam e por que eles funcionam?

## 3.2. Uma (extremamente) breve reflexão sobre complexidade dos algoritmos

Quanto custa para calcular a variância (no caso, desvio padrão) e o percentil nos exemplos anteriores?

**Modo batch**

Variância:  $\dfrac{\sum_i^N (x_i - \bar{x})}{N - 1}$

- Média é calculada com o custo $O(n)$ -> todos os elementos devem ser somados e divididos pelo total de observações
- Com a média em mãos, precisamos calcular o desvio de cada elemento para a média e somar tais desvios: $O(n)$
- No fim das contas, o custo final é $O(n)$ (na notação assintótica), mas tivemos que percorrer todos os dados duas vezes (e, consequentemente, armazená-los).
- Esse algoritmo também apresenta problemas de estabilidade, quando o número de observações é muito grande. Pode gerar erros numéricos de aproximação.

Percentil:

- Pegue um exemplo novo e remova o mais antigo
- Ordene os dados: algo em torno de $O(n\log n)$ (n é o tamanho do buffer)
- Encontre a posição correta, interpole e retorne

Se esse processo é repetido várias e várias vezes...

**Modo incremental**


Variância (algoritmo de Welford):

- Precisamos de algumas variáveis:
    - $n$: número de observações
    - $\overline{x}_n$: a média da amostral, após $n$ observações
    - $M_{2, n}$: estatística de segunda ordem que gerará a variância
- As atualizações das estatísticas se dão na seguinte forma:
    - $\overline{x}_n = \overline{x}_{n-1} + \dfrac{x_n - \overline{x}_{n-1}}{n}$
    - $M_{2,n} = M_{2,n-1} + (x_n - \overline{x}_{n-1})(x_n - \overline{x}_n)$
- E as estatísticas são inicializadas da seguinte forma:
    - $\overline{x}_{0} \leftarrow 0$
    - $M_{2,0} \leftarrow 0$
- Para obter a variância amostral basta usar: $s_n^2 = \dfrac{M_{2,n}}{n-1}$, para todo $n > 1$
- De brinde, obtemos um preditor robusto para a média 🤓


Percentil:

- Mantemos dois buffers
   - Um com os dados na ordem em que chegam
   - Outro com os dados ordenados: o custo de inserção de um novo ponto em um vetor já ordenado é $O(\log n)$
   - A remoção de um valor no buffer ordenado é $O(n)$. No buffer não-ordenado é $O(1)$
   - O cálculo do percentil usa o buffer ordenado
   
No fim das contas, sempre buscamos que cada atualização em uma métrica ou modelo de aprendizado tenha um custo constante ($O(1)$). Se isso não for possível, então um custo sub-linear é o objetivo!

---

Antes de prosseguirmos, vou mostrar mais uma aplicação legal de métricas incrementais, com requintes de processamento distribuido.

**Situação hipotética:**

E se estivéssemos calculando estatísticas de alguma coisa, mas a coleta era feita de forma separada? Por exemplo, estamos calculando a variância de alguma coisa a nível estadual, mas a coleta é feita por município?

(Eu sou paranaense)

In [14]:
rng = random.Random(7)

In [17]:
dados_ca = [rng.gauss(1500, 200) for _ in range(15000)]
dados_lndrn = [rng.gauss(2500, 500) for _ in range(600000)]

np.var(dados_ca), np.var(dados_lndrn)

(39670.63602877245, 250112.88479177756)

E se quisermos calcular a variância (ou média) total de todos os municípios?

In [18]:
dados_parana = []

# dados_parana.extend(dados_abatiá)
# ...
dados_parana.extend(dados_ca)
# ...
dados_parana.extend(dados_lndrn)
# ...
# dados_parana.extend(dados_xambrê)

len(dados_parana)

615000

In [21]:
np.var(dados_parana)

268873.83428214735

In [20]:
np.mean(dados_parana)

2475.850834856662

Nós já sabemos como fazer esse mesmo processo de forma incremental:

In [22]:
# estou considerando que todos os dados foram coletados, mas poderia usar o padrão, ddof=1
var_ca = stats.Var(ddof=0)
var_lndrn = stats.Var(ddof=0)

for p in dados_ca:
    var_ca.update(p)
    
for p in dados_lndrn:
    var_lndrn.update(p)

In [23]:
var_ca.get(), var_lndrn.get()

(39670.63602877236, 250112.88479177424)

Posso usar `stats.Mean` para obter a média. Mas se lembrarmos bem, o algoritmo de Welford tem um preditor de média "lá no meio", não tem?

Voilà!

In [25]:
var_ca.mean.get(), np.mean(dados_ca)

(1498.2274459163768, 1498.227445916374)

Agora vem a parte mais legal 🙃

Como obtemos as estatísticas para o estado inteiro?

In [27]:
# Imagine que temos um for somando as estatísticas de todos os estados

var_parana = var_ca + var_lndrn  # + os outros municipios

var_parana.get(), np.var(dados_parana)

(268873.83428214834, 268873.83428214735)

In [28]:
var_parana.mean.get(), np.mean(dados_parana)

(2475.8508348567498, 2475.850834856662)

E se a bela e formosa cidade de Cândido de Abreu tivesse sido esquecida? Os dados tivessem sido perdidos, sei lá...

(quando eu era criança, minha cidade nem aparecia nos mapas impressos do Paraná, vai saber...)

In [29]:
var_ca_backup = var_parana - var_lndrn  # - os outros municipios

In [30]:
var_ca.get(), var_ca_backup.get()

(39670.63602877236, 39670.63602878291)

In [31]:
var_ca.mean.get(), var_ca_backup.mean.get()

(1498.2274459163768, 1498.2274459163825)

No fim das contas, sempre existe um trade-off entre memória e tempo de processamento (e a quantidade de "passadas" nos dados) e a precisão dos resultados obtidos. Muitos dos algorítmos de online learning incluem um parâmetro ($\delta$) que indica a porcentagem de "certeza" que você deseja ter nos seus resultados. Em geral, quanto mais certeza, mais tempo leva para tomar as decisões.


# 4. Por que usar dicionários (ou, por que usar uma representação esparsa)?

# 5. Como avaliar um modelo?

# 6. Exemplos de algoritmos

## 6.1. Classificação

## 6.2. Regressão

## 6.3. Clustering

## 6.4. Anomaly detection

# 7. Expert stuff

# 8. Pipelines e pré-processamento

# 9. Exemplo completo